# ポイントクラウドを用いた平面の検出


Q21: ベクトル $\bf a$ のノルム（長さ）を $\|\bf a\|$と記す．また，ベクトル $\bf a$ と $\bf b$の内積を ${\bf a}\cdot{\bf b}$，外積を ${\bf a}\times{\bf b}$ と記す．__これらの記法を用いて__，「位置ベクトル ${\bf p}_0$，${\bf p}_1$，${\bf p}_2$ の3点を通る平面」と「位置ベクトル $\bf p$ の点」の間の距離 $d$ を求める公式を作れ．

Q22: Q21の公式を図を用いて解説せよ．

## RealSenseで平面を撮像してください．
> 机や壁と多面体など，なるべく2つ以上の面が広く写るように撮影しましょう．

> このファイルの内容は，★印まで pointcloud_rs.ipynb と同じです．★印まで同様に作業を進めてください． 

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
from IPython.display import Image, display
import matplotlib.pyplot as plt
import mediapipe as mp
%matplotlib inline

# Configure color and depth to run at VGA resolution at 30 frames per second
config = rs.config()
config.enable_stream(rs.stream.depth)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)

## カラー画像と深度画像を取得して表示・保存します．
スペースキーを押す毎に画像が連番で保存されます．'q'を押すと終了します．最後に保存したデータが点群の作成に使われます．

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

# Start streaming
pipeline = rs.pipeline()
profile = pipeline.start(config)

# Get camera parameters
intr = profile.get_stream(rs.stream.depth).as_video_stream_profile().get_intrinsics()
scale = config.resolve(rs.pipeline_wrapper(pipeline)).get_device().first_depth_sensor().get_depth_scale()

print("focal length(x) in pixels = ", intr.fx)
print("focal length(y) in pixels = ", intr.fy)
print("image height = ", intr.height)
print("image width = ", intr.width)
print("ppx = ", intr.ppx)
print("ppy = ", intr.ppy)

# Create a camera alignment object (depth aligned to color)
align = rs.align(rs.stream.color)
max_depth = 2.0 / scale # Zeros out for any depth greater than 2.0 meters

# Display and save images
print("Press [SPACE] to save images (png) and depth data (npy).")
print("Press 'q' to stop.")
nsaved = 0
try:
    with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:
        while True:
            # Wait for a coherent pair of frames: depth and color
            frames = pipeline.wait_for_frames()
            aligned_frames = align.process(frames)
            color_frame = aligned_frames.get_color_frame()
            depth_frame = aligned_frames.get_depth_frame()
            if not depth_frame or not color_frame:
                continue

            # Convert images to numpy arrays
            bgr = np.asanyarray(color_frame.get_data())
            depth = np.asanyarray(depth_frame.get_data())
            depth[depth > max_depth] = 0 # Zeros out

            # Apply colormap on depth image (image must be converted to 8-bit per pixel first)
            depth_colormap = cv2.applyColorMap(cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U), 
                                               cv2.COLORMAP_JET)

            images = np.hstack((bgr, depth_colormap))
            images.flags.writeable = False
            images = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)
#             results = pose.process(images) # mediapipe
            results_bgr = pose.process(bgr) # mediapipe
            results_depth = pose.process(depth_colormap) # mediapipe
            # Show images
            images.flags.writeable = True
            images = cv2.cvtColor(images, cv2.COLOR_RGB2BGR)
#             mp_drawing.draw_landmarks(
#                 images,
#                 results.pose_landmarks,
#                 mp_pose.POSE_CONNECTIONS,
#                 landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            mp_drawing.draw_landmarks(
                bgr,
                results_bgr.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
            
            mp_drawing.draw_landmarks(
                depth_colormap,
                results_depth.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style()
            )
#             cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
#             cv2.imshow('RealSense', images)
            cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
            cv2.namedWindow('RealSense2', cv2.WINDOW_AUTOSIZE)
            cv2.imshow('RealSense', bgr)
            cv2.imshow('RealSense2', depth_colormap)
            key = cv2.waitKey(33)
            if key == ord(' '):
                Z = depth * scale * 1e+3 # unit in mm
                color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)

                # Save images
                cv2.imwrite('color{:02d}pc.png'.format(qqqqqq), bgr)
                cv2.imwrite('depth{:02d}pc.png'.format(nsaved), depth_colormap)
                np.save('Z{:02d}pc.npy'.format(nsaved), Z)

                print("color image and depth data are saved ({:02d})".format(nsaved))
                nsaved += 1

            elif key == ord('q'):
                if nsaved == 0:
                    Z = depth * scale * 1e+3 # unit in mm
                    color = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
                cv2.destroyAllWindows()
                break
        
finally:
    # Stop streaming
    pipeline.stop()

## 逆透視変換でポイントクラウドを作りましょう．
カラー画像および点群を作成する深度データを可視化します．

In [ ]:
from PIL import Image
height, width, _ = color.shape
plt.figure(figsize=(15,4))
plt.subplot(1,2,1)
# plt.imshow(color)
plt.imshow(color)
plt.subplot(1,2,2)
plt.imshow(Z, cmap="gray")